In [ ]:
import json
import os
import os.path as osp
import pickle as pkl
import time
import warnings

import torch
from lrp_particlenet import LRP_ParticleNet
from model import ParticleNet
from utils import load_data, make_dr_Mij_plots
from fastjet_utils import scaling_up

from torch_geometric.loader import DataLoader

warnings.filterwarnings("ignore")

# Check if the GPU configuration and define the global base device
if torch.cuda.device_count() > 0:
    print("Will use GPU model:", torch.cuda.get_device_name(0))
    device = torch.device("cuda:0")
else:
    print("Will use CPU")
    device = torch.device("cpu")

In [ ]:
# this script runs lrp on a trained ParticleNet model
dataset = "data/toptagging"
outpath = "experiments"
model_prefix = "ParticleNet_dropout1"
epoch = -1  # epoch to run Rscores for (-1=best_epoch, 0=untrained)
quick = False
run_lrp = True
run_dr_Mij_plots = True
run_scaling_up = True

In [ ]:
outpath = osp.join(outpath, model_prefix)

if not os.path.exists(f"{outpath}/xai"):
    os.makedirs(f"{outpath}/xai")

# run lrp pipeline to compute the Rscores
if run_lrp:
    print(f"Runing the LRP pipeline to compute the Rscores for the model at epoch {epoch}")
    # load the testing data
    print("- loading datafiles for lrp studies...")
    data_test = load_data(dataset, "test", 4, quick)
    loader = DataLoader(data_test, batch_size=1, shuffle=True)

    # load a pretrained model
    with open(f"{outpath}/model_kwpkl", "rb") as f:
        model_kwargs = pkl.load(f)

    if epoch == -1:  # load the best trained model
        state_dict = torch.load(f"{outpath}/best_epoch_weights.pth", map_location=device)
        PATH = f"{outpath}/xai/Rscores_best/"
    elif epoch == 0:  # load the untrained model
        state_dict = torch.load(
            f"{outpath}/before_training_weights.pth",
            map_location=device,
        )
        PATH = f"{outpath}/xai/Rscores_untrained/"
    else:  # load a specefic epoch of the trained model
        state_dict = torch.load(
            f"{outpath}/epoch_weights/epoch_{epoch}_weights.pth",
            map_location=device,
        )
        PATH = f"{outpath}/xai/Rscores_epoch_{epoch}/"

    # the following line will make it possible to retrieve intermediate activations
    model_kwargs["for_LRP"] = True

    # instantiate a ParticleNet model with the loaded configuration
    model = ParticleNet(**model_kwargs)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(device)
    print(model)

    # make directory to hold the edgeRscores
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    # initilize an lrp instance
    lrp = LRP_ParticleNet(device=device, model=model, epsilon=1e-8)

    # initilize placeholders to store the input, target, and p4 for each jet
    batch_x_list, batch_y_list, batch_p4_list = [], [], []
    # initilize placeholders to hold the edgeRscores and edge_index of each EdgeConv block for each jet
    R_edges_list, edge_index_list = [], []

    ti = time.time()
    for i, jet in enumerate(loader):
        if i == 5 and quick:
            break

        if i == 1000:
            break

        print(f"Explaining jet # {i}: {jet}")

        # explain a single jet
        try:
            R_edges, edge_index = lrp.explain(jet.to(device))
        except Exception:
            print("jet is not processed correctly so skipping it")
            continue

        batch_x_list.append(jet.x.detach().cpu())
        batch_y_list.append(jet.y.detach().cpu())

        # for fast jet, store the p4 information
        batch_p4_list.append(
            torch.cat(
                [
                    jet.px.detach().cpu().unsqueeze(1),
                    jet.py.detach().cpu().unsqueeze(1),
                    jet.pz.detach().cpu().unsqueeze(1),
                    jet.E.detach().cpu().unsqueeze(1),
                ],
                axis=1,
            )
        )

        R_edges_list.append(R_edges)
        edge_index_list.append(edge_index)

        print("------------------------------------------------------")

    tf = time.time()

    with open(f"{PATH}/time.json", "w") as fp:  # dump time
        json.dump(
            {
                "time_min": round((tf - ti) / 60, 3),
                "time_hours": round((tf - ti) / 3600, 3),
            },
            fp,
        )

    # store the lists containing the jet information
    with open(f"{PATH}/batch_x.pkl", "wb") as handle:
        pkl.dump(batch_x_list, handle)
    with open(f"{PATH}/batch_y.pkl", "wb") as handle:
        pkl.dump(batch_y_list, handle)
    with open(f"{PATH}/batch_p4.pkl", "wb") as handle:
        pkl.dump(batch_p4_list, handle)

    # store the lists containing the edgeRscores and edge_index of each EdgeConv block for each jet
    with open(f"{PATH}/R_edges.pkl", "wb") as handle:
        pkl.dump(R_edges_list, handle)
    with open(f"{PATH}/edge_index.pkl", "wb") as handle:
        pkl.dump(edge_index_list, handle)

    print(f"Finished computing the Rscores for the model at epoch {epoch}")

# produce the lrp deltaR and Mij result plots
if run_dr_Mij_plots:
    print(f"Computing the deltaR and invariant mass distributions of the most relevant edges at epoch {epoch}")
    make_dr_Mij_plots(f"{outpath}/xai", epoch=epoch, Top_N=5)

# produce the lrp scaling_up result plots
if run_scaling_up:
    print(f"Computing the fraction of relevant edges connecting different subjets for the model at epoch {epoch}")
    scaling_up(
        f"{outpath}/xai",
        epoch=epoch,
        N_values=15,
        N_SUBJETS=3,
        JET_ALGO="CA",
        jet_radius=0.8,
    )